### Structure Generation

In [5]:
! pwd

/mounts/Users/cisintern/zeinabtaghavi/MetatagIndexing_2/Dataset_Generation/GeneratingCodes/S_Uni


In [63]:
import ast
import json
import random

with open("../../Dataset_Helping/names.txt", "r") as file:
    names = ' '.join([line.strip() for line in file.readlines()])
    names_list = sorted(list(ast.literal_eval(names)))

# Load the shopping data from the JSON file.
# The file should have the structure as you described.
with open("../../Dataset_Helping/S_Both/Wikidata_Final.jsonl", "r") as f:
    wikidata_samples = [json.loads(line) for line in f]


In [64]:
# untill you keep the country as a label
for sample in wikidata_samples:
    if 'country' not in sample.keys():
        country = sample['label'].split('-')[-1].strip()
        sample['country'] = country

In [65]:
wikidata_samples[0]

{'item': 'Nkrumah University',
 'label': 'Kabwe - Zambia',
 'score': 0.17619562149047852,
 'c4_count': 8527,
 'country': 'Zambia'}

In [86]:
import random
def generate_user_trip_destination(country_list, num_items):
    country_list_names = sorted(list(country_list.keys()))
    country_list_names = random.sample(country_list_names, num_items)
    user_trip_destinations = []
    for country in country_list_names:
        user_trip_destinations.append(random.choice(country_list[country]))

    return user_trip_destinations


In [87]:
import random
random.seed(42)

user_num = 50
num_items = 30

users = random.sample(names_list, user_num)

country_list = {}
for sample in wikidata_samples:
    if sample['country'] not in country_list.keys():
        country_list[sample['country']] = [sample]
    else:
        country_list[sample['country']].append(sample)

user_dataset = {}

for user in users:
    user_dataset[user] = {"user": user, "trip_info": []}

    user_trips_items = generate_user_trip_destination(country_list, num_items)

    friends = random.sample([f for f in names_list if f != user], int(num_items * 2))
    users_2 = random.sample(friends,num_items)
    trip_friends = [f for f in friends if f not in users_2]
    for i in range(num_items):
        user_2 = users_2[i]
        trip = user_trips_items[i]
        trip_friend = trip_friends[i]

        trip_info = {
            "user_2": user_2,
            "trip_destination": trip['item'],
            "trip_friends": trip_friend,
            "trip_country": trip['country'],
        }
        user_dataset[user]["trip_info"].append(trip_info)
# user_dataset





In [89]:
# Save the dataset to a JSON file
output_file = '../../Dataset_Helping/S_Uni/S_Uni_Structured.jsonl'
with open(output_file, 'w') as f:
    for user, data in user_dataset.items():
        line = {'user': user, 'trip_info': data['trip_info']}
        f.write(json.dumps(line) + '\n')


### Merging

In [ ]:
import random
from datetime import timedelta, date, datetime
import ast
from pathlib import Path
import json
import os
# ------------------------------------------------------------
# Initialize random seed
# ------------------------------------------------------------

random.seed(42)

def merging_dataset(base_path):

    # Load the structured data
    structured_data_path = base_path + "Dataset_Helping/S_Uni/S_Uni_Structured.jsonl"

    with open(structured_data_path, 'r', encoding='utf-8') as f:
        structured_data = [json.loads(line) for line in f]

    print(f"Loaded {len(structured_data)} records from structured data file")

    # Load the generated conversation data
    generated_data_path = base_path + "Dataset_Helping/S_Uni/S_Uni_Structured_Generated_conversation.jsonl"

    with open(generated_data_path, 'r', encoding='utf-8') as f:
        generated_data = [json.loads(line) for line in f]

    print(f"Loaded {len(generated_data)} records from generated data file")

    dataset = []


    x = 0
    for i in range(len(structured_data)):
        print(structured_data[i])

        user_ID = i
        user = structured_data[i]['user']
        trip_info_list = structured_data[i]['trip_info']

        for j in range(len(trip_info_list)):
            user_2 = trip_info_list[j]['user_2']
            if i*30 + j != x:
                print(i*30 + j, x)
            assert i*30 + j == x
            x += 1
            conversation = generated_data[int(i*20 + j)]
            conversation = ast.literal_eval(conversation.replace('\\', ''))


            hour = random.randint(8, 17)
            minute = sorted(random.sample(range(0, 60), 10))

            # user_response = generated_data[int(i*20 + j)]
            if conversation == '-':
                print(conversation, i*20 + j)
                raise Exception("conversation is '-'")

            conversation = [(f"{conversation[s][0]} {hour:02d}:{minute[s]:02d}", conversation[s][1], conversation[s][2]) for s in range(len(conversation))]


            question = f"What did {user} go to {trip_info_list[j]['trip_country']} with?"
            dataset.append({
                "user_ID": user_ID,
                "user": user,
                "user_2": user_2,
                "conversation": conversation,
                "extra_info": {k:v for k,v in trip_info_list[j].items() if k in ['type_of_location']},
                "question": question,
                "answer": trip_info_list[j]['trip_friends']
            })

    with open(base_path + '/Data/S_Uni.jsonl', 'w', encoding='utf-8') as f:
        for item in dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')
    print(f"S_Uni: {len(dataset)}, stored in {base_path}/Data/S_Uni.jsonl")
if __name__ == "__main__":

    base_path = '../../../Dataset_Generation/'
    merging_dataset(base_path)

Loaded 50 records from structured data file
Loaded 1500 records from generated data file
{'user': 'Elita', 'trip_info': [{'user_2': 'Creighton', 'trip_destination': 'Ictus Theatre', 'trip_friends': 'Sigil', 'trip_country': 'Chile', 'type_of_location': ['building', 'venue']}, {'user_2': 'Guinlan', 'trip_destination': 'Laudation of the Virgin church at Ratmino', 'trip_friends': 'Weiland', 'trip_country': 'Russia', 'type_of_location': ['church building', 'architectural landmark']}, {'user_2': 'Hachiko', 'trip_destination': "Jong's Crocodile Farm and Zoo", 'trip_friends': 'Allegria', 'trip_country': 'Malaysia', 'type_of_location': ['zoo', 'crocodile farm']}, {'user_2': 'Raleigh', 'trip_destination': 'Stanga House', 'trip_friends': 'Lochlann', 'trip_country': 'Switzerland', 'type_of_location': ['museum building', 'architectural structure']}, {'user_2': 'Silvin', 'trip_destination': 'Rates Ecomuseum', 'trip_friends': 'Odette', 'trip_country': 'Portugal', 'type_of_location': ['museum']}, {'us

IndexError: tuple index out of range